# Workshop 1: Machine Learning for Windows
## Step 1: Installing Python
- Navigate to the Windows Store (most easily accessed through the Start Menu)
- Search "Python" in the search bar
Install "Python 3.11"
    - This is not the newest version of python, but this older version is necessary for the packages to be compatible
    - Do not install "Python 3.12"
## Step 2: Setup Python Virtual Environment
- Within VS Code access the terminal at the bottom of the screen by going to the bottom of the main panel until your cursor changes into a up arrow then dragging up
- Once, this panel is revealed, click on the "terminal" tab
- If you already have a terminal type "exit" then hit enter then follow the instructions above
- Now, copy and paste the following commands into the terminal one at a time (you can't use ctrl-v to paste into Powershells. You can just click on the cell and ctrl-c to copy it):

python3.11 -m venv ml-venv

./ml-venv/Scripts/Activate.ps1

- If you run into an error saying that "execution of scripts is disabled on this system, run the following command:
    - Set-ExecutionPolicy RemoteSigned

pip install -r requirements.txt

## Step 3: Test PyTorch and Pandas
- Run the following code segment to verify that the virtual environment is working as intended
    - When doing this, VS Code will prompt you for a kernel at the top of the screen
    - Click on "Python Environments" then "ml-venv"
- If VS Code prompts you to install something when you run the following code block, press "Install"

In [2]:
import torch
import pandas as pd
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt

# Step 4: Importing and Cleaning Data

In [ ]:
# import CSV data located in the data folder through pandas
heartData = pd.read_csv('data/heart.csv')
o2Data = pd.read_csv('data/o2Saturation.csv', header=None)

# copy the O2 saturation data to the main pandas dataframe
heartData['o2Saturation'] = o2Data

# moe the output column to the end of the dataframe
cols = list(heartData.columns)
cols.remove('output')
cols.append('output')
heartData = heartData[cols]

# normalize the dataset between 0 and 1
for col in heartData.columns:
    # these columns are already between 0 and 1 so we don't need to normalize them
    if col == 'output' or col == 'sex' or col == 'fbs' or col == 'exng': continue

    # normalize the data by subtracting the minimum value and dividing by the range
    heartData[col] = (heartData[col] - heartData[col].min()) / (heartData[col].max() - heartData[col].min())

# display the first 5 rows of the dataframe to give an idea of what the data looks like
heartData.head()

# Step 5: Split the Dataset

In [ ]:
# the training set with be 80% of the data, this randomly selects 80% of the data and resets the index
train = heartData.sample(frac=0.8, random_state=42)
train = train.reset_index(drop=True)

# the validation and test set will each be 10% of the data
# after the training set is removed from the data, the remaining data is split in half, one half for validation and the other for testing
validation = heartData.drop(train.index)
test = validation.sample(frac=0.5, random_state=42)
validation = validation.drop(test.index)

# reset the index of the validation and test sets
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

## Step 5.5: Visualizing Connections
- The following code displays the correlation coefficient between each of the 14 variables and the output variable
- As you may notice, none of the variables on their own can accurately predict whether the patient will experience a heart attack
- This is why a holistic analysis of the data using deep learning is advantagous

In [ ]:
plt.figure(figsize=(12, 1))
sns.heatmap(train.corr()[['output']].T, annot=True, cmap='coolwarm', fmt='.2f', cbar=False)
plt.show()

# Step 6: Define a Dataset Object
- To make the data easier to parse through while training, PyTorch offers a Dataset class we can inherant and implement for our own data

In [ ]:
# this will inherit the torch Dataset class and override the __len__ and __getitem__ methods
class HeartDataset(torch.utils.data.Dataset):
    # the constructor will take a dataframe as an argument
    def __init__(self, df):
        self.df = df
    
    # the __len__ method will return the number of rows in the dataframe
    def __len__(self):
        return len(self.df)

    # everytime we parse through the dataset, this method will be called
    def __getitem__(self, idx):
        # get the row at the current
        row = self.df.iloc[idx]
        
        # the input will be all the columns except the output column because the output column is the answer
        x = torch.tensor(row.drop('output').values, dtype=torch.float32)
        
        # the expected output will be the output column
        y = torch.tensor(row['output'], dtype=torch.float32)

        # new need to reshape the tensor from a row vector to a column vector so the answer can be lined up with the output of the model
        y = y.reshape(1)

        # return the input and expected output
        return x, y

# Step 7: Design the Neural Network

In [ ]:
class HeartAttackModel(nn.Module):
    def __init__(self):
        super().__init__()
        # the model will have 3 layers with 64 neurons each
        # the 14 input neurons are the number of columns in the dataset
        self.layer1 = nn.Linear(14, 64)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(64, 64)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(64, 64)
        self.act3 = nn.ReLU()

        # the output layer will have 1 neuron because the output is binary
        self.output = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # the forward method will pass the input through each layer and activation function
        # this is what is called when the model is used
        x = self.layer1(self.act1(x))
        x = self.layer2(self.act2(x))
        x = self.layer3(self.act3(x))
        x = self.sigmoid(self.output(x))
        return x

# Step 8: Create Training Function

In [ ]:
def train_model(model, train_loader, val_loader, epochs, lr):
    # the model will be trained using the binary cross entropy loss function and the Adam optimizer
    # the loss function will be used to determine how well the model is performing
    criterion = nn.BCELoss()
    
    # the optimizer will be used to adjust the weights of the model
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # the model will be trained for the number of epochs specified
    for epoch in range(epochs):
        # turns the model to training mode
        model.train()

        # loops through all the data in the training set
        for x, y in train_loader:
            # get the output of the model
            y_pred = model(x)

            # calculate the loss
            loss = criterion(y_pred, y)

            # zero the gradients
            optimizer.zero_grad()

            # calculate the gradients
            loss.backward()

            # update the weights
            optimizer.step()

        # turn the model to evaluation mode
        model.eval()

        # turn off gradient tracking
        with torch.no_grad():
            # the loss will be calculated on the validation set and printed
            val_loss = 0
            for x, y in val_loader:
                y_pred = model(x)
                loss = criterion(y_pred, y)
                val_loss += loss.item()
            print(f'Epoch {epoch} - Validation Loss: {val_loss}')

# Step 9: Initialize Objects and Train Model

In [ ]:
# create the model and datasets
model = HeartAttackModel()
train_dataset = HeartDataset(train)
val_dataset = HeartDataset(validation)

# format the data for the model
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False)

# train the model
train_model(model, train_loader, val_loader, 100, 0.001)

# Step 10: Test Model Performance

In [ ]:
# create the test dataset and loader and test the model
test_dataset = HeartDataset(test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False)
model.eval()

# turn off gradient tracking
with torch.no_grad():
    correct = 0
    for x, y in test_loader:
        y_pred = model(x)
        y_pred = torch.round(y_pred)
        print(y_pred[0][0].item(), y[0][0].item())
        correct += (y_pred == y).sum().item()
    # print the accuracy of the model
    print(f'Accuracy: {round((correct / (len(test_loader)*2))*100, 3)}%')

Link to dataset: https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset